## Votação Seção


```mysql
create table votacao_secao_2012(
    DATA_GERACAO STRING,
    HORA_GERACAO STRING,
    ANO_ELEICAO STRING,
    NUM_TURNO STRING,
    DESCRICAO_ELEICAO STRING,
    SIGLA_UF STRING,
    SIGLA_UE STRING,
    CODIGO_MUNICIPIO STRING,
    NOME_MUNICIPIO STRING,
    NUM_ZONA STRING,
    NUM_SECAO STRING,
    CODIGO_CARGO STRING,
    DESCRICAO_CARGO STRING,
    NUM_VOTAVEL STRING,
    QTDE_VOTOS STRING) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
'separatorChar' = '\;',
'quoteChar'     = '\"',
'serialization.encoding'='ISO-8859-1'
)
STORED AS TEXTFILE;
Alter table votacao_secao_2012 set location 's3://datascience2016/data/gzip/votacao_secao/2012/';
```



## Candidato

```mysql
create table candidato_2012 
    (DATA_GERACAO STRING,
    HORA_GERACAO STRING,
    ANO_ELEICAO STRING,
    NUM_TURNO STRING,
    DESCRICAO_ELEICAO STRING,
    SIGLA_UF STRING,
    SIGLA_UE STRING,
    DESCRICAO_UE STRING,
    CODIGO_CARGO STRING,
    DESCRICAO_CARGO STRING,
    NOME_CANDIDATO STRING,
    SEQUENCIAL_CANDIDATO STRING,
    NUMERO_CANDIDATO STRING,
    CPF_CANDIDATO STRING,
    NOME_URNA_CANDIDATO STRING,
    COD_SITUACAO_CANDIDATURA STRING,
    DES_SITUACAO_CANDIDATURA STRING,
    NUMERO_PARTIDO STRING,
    SIGLA_PARTIDO STRING,
    NOME_PARTIDO STRING,
    CODIGO_LEGENDA STRING,
    SIGLA_LEGENDA STRING,
    COMPOSICAO_LEGENDA STRING,
    NOME_LEGENDA STRING,
    CODIGO_OCUPACAO STRING,
    DESCRICAO_OCUPACAO STRING,
    DATA_NASCIMENTO STRING,
    NUM_TITULO_ELEITORAL_CANDIDATO STRING,
    IDADE_DATA_ELEICAO STRING,
    CODIGO_SEXO STRING,
    DESCRICAO_SEXO STRING,
    COD_GRAU_INSTRUCAO STRING,
    DESCRICAO_GRAU_INSTRUCAO STRING,
    CODIGO_ESTADO_CIVIL STRING,
    DESCRICAO_ESTADO_CIVIL STRING,
    CODIGO_COR_RACA STRING,
    DESCRICAO_COR_RACA STRING,
    CODIGO_NACIONALIDADE STRING,
    DESCRICAO_NACIONALIDADE STRING,
    SIGLA_UF_NASCIMENTO STRING,
    CODIGO_MUNICIPIO_NASCIMENTO STRING,
    NOME_MUNICIPIO_NASCIMENTO STRING,
    DESPESA_MAX_CAMPANHA STRING,
    COD_SIT_TOT_TURNO STRING,
    DESC_SIT_TOT_TURNO STRING,
    NM_EMAIL STRING)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
'separatorChar' = '\;',
'quoteChar'     = '\"',
'serialization.encoding'='ISO-8859-1'
)
STORED AS TEXTFILE;
Alter table candidato_2012 set location 's3://datascience2016/data/gzip/candidatos/candidato_2012.csv.gz';
```


### Legenda


```mysql
create table legenda_2012(
    DATA_GERACAO STRING,
    HORA_GERACAO STRING,
    ANO_ELEICAO STRING,
    NUM_TURNO STRING,
    DESCRICAO_ELEICAO STRING,
    SIGLA_UF STRING,
    SIGLA_UE STRING,
    NOME_UE STRING,
    CODIGO_CARGO STRING,
    DESCRICAO_CARGO STRING,
    TIPO_LEGENDA STRING,
    NUM_PARTIDO STRING,
    SIGLA_PARTIDO STRING,
    NOME_PARTIDO STRING,
    SIGLA_COLIGACAO STRING,
    NOME_COLIGACAO STRING,
    COMPOSICAO_COLIGACAO STRING,
    SEQUENCIA_COLIGACAO STRING)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
'separatorChar' = '\;',
'quoteChar'     = '\"'
)
STORED AS TEXTFILE;
Alter table legenda_2012 set location 's3://datascience2016/data/gzip/legendas/legendas_2012.csv.gz';
```





## Candidato - legenda
```mysql
    select 
    * 
    from candidato_2012 as c 
    join legenda_2012 as l 
    on c.SIGLA_UE=l.SIGLA_UE 
    AND c.NUMERO_PARTIDO=l.NUM_PARTIDO 
    AND c.CODIGO_CARGO=l.CODIGO_CARGO
```

# Agregações Regionais


### macro
```mysql
create view votacao_secao_2012_macro as
select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
SIGLA_UE,
CODIGO_MACRO,
NOME_MACRO,
sum(cast(QTDE_VOTOS AS INT)) as QTDE_VOTOS
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO
from votacao_secao_2012 v join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by 
ANO_ELEICAO,
NUM_TURNO,
SIGLA_UE,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO;

```

### UF
```mysql
create view votacao_secao_2012_uf as
select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
SIGLA_UE,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF,
sum(cast(QTDE_VOTOS AS INT)) as QTDE_VOTOS
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO,m.UF,m.NOME_UF 
from votacao_secao_2012 v join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
SIGLA_UE,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF;

```
### meso

```mysql
create view votacao_secao_2012_meso as
select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
SIGLA_UE,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF,
CODIGO_MESO,
NOME_MESO,
sum(cast(QTDE_VOTOS AS INT)) as QTDE_VOTOS
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO,m.UF,m.NOME_UF,m.CODIGO_MESO,m.NOME_MESO 
from votacao_secao_2012 v join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
NOME_MACRO,
SIGLA_UE,
UF,
NOME_UF,
CODIGO_MESO,
NOME_MESO;
```
### micro

```mysql
create view votacao_secao_2012_micro as
select 
ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
SIGLA_UE,
CODIGO_MACRO,
NOME_MACRO,
UF,
NOME_UF,
CODIGO_MESO,
NOME_MESO,
CODIGO_MICRO,
NOME_MICRO,
sum(cast(QTDE_VOTOS AS INT)) as QTDE_VOTOS
from (
select v.*,
m.CODIGO_MACRO,m.NOME_MACRO,m.UF,m.NOME_UF,m.CODIGO_MESO,m.NOME_MESO,m.CODIGO_MICRO,m.NOME_MICRO 
from votacao_secao_2012 v join aux_municipio as m on m.CODIGO_MUNICIPIO=v.CODIGO_MUNICIPIO
) as v_s_m group by ANO_ELEICAO,
NUM_TURNO,
CODIGO_CARGO,
NUM_VOTAVEL,
CODIGO_MACRO,
UF,
NOME_UF,
SIGLA_UE,
NOME_MACRO,
CODIGO_MESO,
NOME_MESO,
CODIGO_MICRO,
NOME_MICRO;

```




## join com os candidatos

```mysql

create view votacao_secao_2012_macro_candidato as
select
    c.ANO_ELEICAO,
    c.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    c.SIGLA_UE,
    c.DESCRICAO_UE ,
    c.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    c.NUMERO_CANDIDATO ,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    c.CODIGO_COR_RACA,
    c.DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    c.NM_EMAIL,
    v.CODIGO_MACRO,
    v.NOME_MACRO,
    v.QTDE_VOTOS
from votacao_secao_2012_macro as v
join candidato_2012 as c
on v.SIGLA_UE=c.SIGLA_UE
AND v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;
create view votacao_secao_2012_uf_candidato as
select
    c.ANO_ELEICAO,
    c.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    c.SIGLA_UE,
    c.DESCRICAO_UE ,
    c.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    c.NUMERO_CANDIDATO ,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    c.CODIGO_COR_RACA,
    c.DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    c.NM_EMAIL,
    v.CODIGO_MACRO,
    v.NOME_MACRO,
    v.UF,
    v.NOME_UF,
    v.QTDE_VOTOS
from votacao_secao_2012_uf as v
join candidato_2012 as c
on v.SIGLA_UE=c.SIGLA_UE
AND v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;


create view votacao_secao_2012_uf_candidato as
select
    c.ANO_ELEICAO,
    c.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    c.SIGLA_UE,
    c.DESCRICAO_UE ,
    c.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    c.NUMERO_CANDIDATO ,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    c.CODIGO_COR_RACA,
    c.DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    c.NM_EMAIL,
    v.CODIGO_MACRO,
    v.NOME_MACRO,
    v.UF,
    v.NOME_UF,
    v.QTDE_VOTOS
from votacao_secao_2012_uf as v
join candidato_2012 as c
on v.SIGLA_UE=c.SIGLA_UE
AND v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;



create view votacao_secao_2012_meso_candidato as
select
    c.ANO_ELEICAO,
    c.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    c.SIGLA_UE,
    c.DESCRICAO_UE ,
    c.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    c.NUMERO_CANDIDATO ,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    c.CODIGO_COR_RACA,
    c.DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    c.NM_EMAIL,
    v.CODIGO_MACRO,
    v.NOME_MACRO,
    v.CODIGO_MESO,
    v.NOME_MESO,
    v.UF,
    v.NOME_UF,
    v.QTDE_VOTOS
from votacao_secao_2012_meso as v
join candidato_2012 as c
on v.SIGLA_UE=c.SIGLA_UE
AND v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;


create view votacao_secao_2012_micro_candidato as
select
    c.ANO_ELEICAO,
    c.NUM_TURNO,
    c.DESCRICAO_ELEICAO,
    c.SIGLA_UE,
    c.DESCRICAO_UE ,
    c.CODIGO_CARGO ,
    c.DESCRICAO_CARGO ,
    c.NOME_CANDIDATO ,
    c.SEQUENCIAL_CANDIDATO ,
    c.NUMERO_CANDIDATO ,
    c.CPF_CANDIDATO ,
    c.NOME_URNA_CANDIDATO ,
    c.COD_SITUACAO_CANDIDATURA ,
    c.DES_SITUACAO_CANDIDATURA ,
    c.NUMERO_PARTIDO ,
    c.SIGLA_PARTIDO ,
    c.NOME_PARTIDO ,
    c.CODIGO_LEGENDA ,
    c.SIGLA_LEGENDA ,
    c.COMPOSICAO_LEGENDA ,
    c.NOME_LEGENDA ,
    c.CODIGO_OCUPACAO,
    c.DESCRICAO_OCUPACAO,
    c.DATA_NASCIMENTO,
    c.NUM_TITULO_ELEITORAL_CANDIDATO,
    c.IDADE_DATA_ELEICAO,
    c.CODIGO_SEXO,
    c.DESCRICAO_SEXO,
    c.COD_GRAU_INSTRUCAO,
    c.DESCRICAO_GRAU_INSTRUCAO,
    c.CODIGO_ESTADO_CIVIL,
    c.DESCRICAO_ESTADO_CIVIL,
    c.CODIGO_COR_RACA,
    c.DESCRICAO_COR_RACA,
    c.CODIGO_NACIONALIDADE,
    c.DESCRICAO_NACIONALIDADE,
    c.SIGLA_UF_NASCIMENTO,
    c.CODIGO_MUNICIPIO_NASCIMENTO,
    c.NOME_MUNICIPIO_NASCIMENTO,
    c.DESPESA_MAX_CAMPANHA,
    c.COD_SIT_TOT_TURNO,
    c.DESC_SIT_TOT_TURNO,
    c.NM_EMAIL,
    v.CODIGO_MACRO,
    v.NOME_MACRO,
    v.UF,
    v.NOME_UF,
    v.CODIGO_MESO,
    v.NOME_MESO,
    v.CODIGO_MICRO,
    v.NOME_MICRO,
    v.QTDE_VOTOS
from votacao_secao_2012_micro as v
join candidato_2012 as c
on v.SIGLA_UE=c.SIGLA_UE
AND v.ANO_ELEICAO=c.ANO_ELEICAO 
AND v.NUM_TURNO=c.NUM_TURNO
AND v.CODIGO_CARGO=c.CODIGO_CARGO
AND v.NUM_VOTAVEL=c.NUMERO_CANDIDATO;

select count(*) from votacao_secao_2012_macro_candidato;
select count(*) from votacao_secao_2012_uf_candidato;
select count(*) from votacao_secao_2012_meso_candidato;
select count(*) from votacao_secao_2012_micro_candidato;

```